# Dependecies and data load

In [1]:
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.graph_objects as go2
import plotly.plotly as py

# conda install -c plotly plotly=4.9.0
# conda install "notebook>=5.3" "ipywidgets>=7.2"

ImportError: 
The plotly.plotly module is deprecated,
please install the chart-studio package and use the
chart_studio.plotly module instead. 


In [2]:
context = pd.read_csv('./data/contextMinJusticia.csv', sep=';')
inmate = pd.read_excel('./data/ReincidenciaPospenadosNal201011Junio2020Rev.xlsx', skiprows = 6)

# Context plotly graphs

In [11]:
context

,Year,Capacity,Population
0,1991,28.319,29.355
1,1992,28.294,27.016
2,1993,28.084,28.260
3,1994,NaN,29.338
4,1995,NaN,31.960
5,1996,28.332,38.063
6,1997,29.239,41.405
7,1998,33.009,43.259
8,1999,33.090,46.322
9,2000,35.969,49.816


In [12]:
people = inmate.groupby(['INTERNOEN','FECHA_INGRESO']).count().reset_index()
people.FECHA_INGRESO = pd.to_datetime(people.FECHA_INGRESO)
people['Year'] = people.FECHA_INGRESO.dt.year
people = people[['INTERNOEN','FECHA_INGRESO','ESTADO','Year']]
people.columns = ['INTERNOEN','FECHA_INGRESO','reoffenders','Year']
#people.reset_index(inplace=True)
people.shape
people = people.groupby('Year').count()
people = people['reoffenders']
people = people.to_frame()
people

,reoffenders
Year,
2010,7475
2011,9584
2012,7948
2013,8972
2014,8232
2015,10211
2016,10789
2017,11133
2018,12433


In [13]:
df_merge = pd.merge(context,people, how='left',on ='Year')
df_merge.set_index('Year',inplace=True)
df_merge['reoffenders'] = df_merge['reoffenders']/1000
df_merge

,Capacity,Population,reoffenders
Year,,,
1991,28.319,29.355,NaN
1992,28.294,27.016,NaN
1993,28.084,28.260,NaN
1994,NaN,29.338,NaN
1995,NaN,31.960,NaN
1996,28.332,38.063,NaN
1997,29.239,41.405,NaN
1998,33.009,43.259,NaN
1999,33.090,46.322,NaN


In [14]:
import plotly.express as px
df = df_merge.reset_index()
temp = pd.melt(df, id_vars=['Year'], value_vars=['Capacity', 'Population', 'reoffenders'])
fig = px.line(temp, x="Year", y="value",line_group="variable", hover_name="variable")
fig.show()

In [15]:
labels = ['Population', 'Capacity', 'reoffenders']
x_data_temp =[]
y_data_temp = []
for i,var in enumerate(labels):
    x_data_temp.append(temp[temp.variable==var]['Year'].values)
    y_data_temp.append(temp[temp.variable==var]['value'].values)
#Key data


In [90]:
import plotly.graph_objects as go
import numpy as np
title = 'Jail Population in Colombia'
labels = ['Population', 'Capacity', 'Reoffenders']
colors = ['rgb(67,67,67)', 'rgb(168, 50, 50)', 'rgb(49,130,189)']
percentages = ['149','100','11']
mode_size = [10, 10, 10]
line_size = [3, 3, 5]
x_data = x_data_temp
y_data = y_data_temp
fig = go.Figure()

for i in range(0, 3):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))
    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=11,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=True,
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)
annotations = []
# Adding labels
for y_trace, label, color, percentage in zip(y_data, labels, colors, percentages):
    annotations.append(dict(xref='paper', x=0.75, y=y_trace[-1],
                                  xanchor='left', yanchor='bottom',
                                  text=label+': {}k, ({}%)'.format(int(y_trace[-1]),percentage),
                                  font=dict(family='Work Sans',
                                            size=14.5),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.3, y=1,
                              xanchor='left', yanchor='bottom',
                              text=title,
                              font=dict(family='Work Sans',
                                        size=25,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Source: INPEC & datos.gov.co',
                              font=dict(family='Work Sans',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

In [97]:
title = 'Jail Population in Colombia'
labels = ['Population', 'Capacity', 'Reoffenders']
colors = ['rgb(194, 197, 204)', 'rgb(194, 197, 204)', 'rgb(102,102,102)']
percentages = ['149','100','11']
mode_size = [10, 10, 10]
line_size = [2, 2, 4]
x_data = x_data_temp
y_data = y_data_temp
fig = go.Figure()

for i in range(0, 3):
    fig.add_trace(go.Scatter(x=x_data[i], y=y_data[i], mode='lines',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,fill='tozeroy',
    ))
    # endpoints
    fig.add_trace(go.Scatter(
        x=[x_data[i][0], x_data[i][-1]],
        y=[y_data[i][0], y_data[i][-1]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))
fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=11,
            color='rgb(82, 82, 82)', 
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
    ),
    autosize=True,
    margin=dict(
        autoexpand=True,
        l=100,
        r=50,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor='white'
)
annotations = []
# Adding labels
for y_trace, label, color, percentage in zip(y_data, labels, colors, percentages):
    annotations.append(dict(xref='paper', x=0.85, y=y_trace[-1],
                                  xanchor='left', yanchor='bottom',
                                  text=label+':{}k'.format(int(y_trace[-1])),
                                  font=dict(family='Work Sans',
                                            size=14.5),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.3, y=1,
                              xanchor='left', yanchor='bottom',
                              text='',
                              font=dict(family='Work Sans',
                                        size=25,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Source: INPEC & datos.gov.co',
                              font=dict(family='Arial Sans',
                                        size=12,
                                        color='rgb(100,100,100)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

In [17]:
number_captures_by_inmate = inmate.groupby(['INTERNOEN'])['FECHA_INGRESO'].nunique().value_counts() \
                            .reset_index().rename(columns = {'index': 'Unique Fecha Ingreso', 
                                                             'FECHA_INGRESO': 'Number of Inmates'})
number_captures_by_inmate

,Unique Fecha Ingreso,Number of Inmates
0,1,58739
1,2,16478
2,3,3023
3,4,497
4,5,91
5,6,10
6,7,2
7,9,1


In [18]:
import plotly.express as px
fig = px.bar(number_captures_by_inmate, 
             x='Unique Fecha Ingreso', 
             y='Number of Inmates',
             title = 'Inmates recividisim count')
fig.show()